## CODIGO PRINCIPAL

En este notebook se incluyen las ejecuciones de los programas desarrollados en el proyecto de forma secuencial.

En concreto este código está dedicado a los capítulos del trabajo indicados a continuación:

    3. Sistema, diseño y desarrollo
    4. Experimentos realizados y resultados

#### Importar funciones y módulos requeridos

El módulo *nimgenetics.py* contiene las funciones necesarias para ejecutar este código

In [ ]:
import os
import sys

sys.path.append('./my_module/')
import nimgenetics

### 3. SISTEMA, DISEÑO Y DESARROLLO

     
Directorios importantes:
     
    - folder_path: carpeta donde se ubican los informes de NIMGenetics.
    - folder_result: carpeta donde se irán guardando los resultados de las ejecuciones de los programas.
    - pathI: path de la carpeta "Identificados" de los informes que cumplen el patrón de selección.
    - image_path: carpeta donde se almacenan las imágenes generadas a partir de los informes.
    - textos: carpeta donde se almacenan los textos generados a partir de las imágenes.

In [ ]:
folder_path = 'path_to_folder_root'
folder_result = 'path_to_folder_result'
pathI = 'path_to_folder_identificados'
image_path = 'path_to_images'

    - 3.2 Obtención de informes clínicos de Gestlab
      Informes filtrados y renombrados
    
        - 3.2.1 Mapeo de los informes localizados en folder_path: load_files.
          La función load_files es utilizada para mapear los informes (PDF) dentro de folder_path.


In [ ]:
# l_path contiene el path absoluto de todos los informes
l_path = nimgenetics.load_files(root_dir=folder_path)[0]
# l_pdf es el mapeo individual de cada informe
l_pdf = nimgenetics.load_files(root_dir=folder_path)[1]

        - 3.2.2 Obtención de los identificadores reales de los informes: getIDs.
          Consulta sql a gestlab para obtener los identificadores reales.

In [ ]:
# id_peticion es una lista con los ID reales obtenidos tras la decodificación hexadecimal
id_peticion = []
for pdf in l_pdf:
    # split por guion bajo
    s_pdf = pdf.split('_')
    # transformación del segundo componente en hexadecimal del identificador
    hex_id = int(s_pdf[1],16)
    id_peticion.append(hex_id)

# ejecución de la función getIDs
r_id = nimgenetics.getIDs(id_list=id_peticion)

        - 3.2.3 Almacenamiento de los informes con su ID correcto: identifiedPDF, emptyPDF.
          Selección de informes que cumplen un patrón específico.

In [ ]:
# Generar una lista que almacene todos los identificadores provenientes de r_id
# l_names a diferencia de la anterior no es una lista de listas
l_names = []   
for row in r_id:
    try:
        # Primer bloque: extraer identificadores que cumplan la siguiente estructura
        l_names.append(row[0][0])
    except:
        # Segundo bloque: identificadores que devuelven elementos vacios
        # Estos informes se almacenarán con su identificador hexadecimal original
        s_row = str(row)
        if s_row == '[]':
            # Almacenamos en l_names los identificadores reemplazados con el literal 'sin_id'
            sr_row = s_row.replace('[]', 'sin_id')
            l_names.append(sr_row)


# Función utilizada para almacenar los informes que cumplen un patrón de selección pre-establecido
# También se almacenan los informes que están repetidos y los descartados que no cumplen el patrón
# con la finalidad de no perder información
nimgenetics.identifiedPDF(folder_path=folder_result, list_id=l_names, list_path=l_path)

# Función para almacenar los informes cuyo identificador no ha sido recuperado de la base de datos
# Estos informes son guardados con el identificador hexadecimal original
nimgenetics.emptyPDF(folder_path=folder_result, list_id=l_names, list_path=l_path, list_pdf=l_pdf)

    - 3.3 Conversión de los informes a texto plano para la extracción de terminología médica

In [ ]:
# Utilizar el módulo os para generar una lista con el contenido del directorio pathI
l_I = os.listdir(pathI)
# pdfPath contiene el path absoluto de todos los pdf la cual es necesaria para ejecutar pdfToImage
pdfPath = []
for pdf in l_I:
    pdfPath.append(os.path.join(pathI, pdf))

        - 3.3.1 Generación de imágenes en formato PNG a partir de los informes escaneados: pdfToImage

In [ ]:
# Ejecución de la función pdfToImage
nimgenetics.pdfToImage(f_path=folder_result, pdf_list=pdfPath)

        - 3.3.2 Generación de texto plano a partir de imágenes PNG: makeCorpus

In [ ]:
# Ejecución de la función makeCorpus la cual integra en su código el programa ocrToImage
# Necesario image_path como parámetro para la correcta ejecución de la función
# El resultado de este proceso guarda los resultados en la carpeta textos
nimgenetics.makeCorpus(f_path=folder_result, image_path=image_path)

### 4. EXPERIMENTOS REALIZADOS Y RESULTADOS

Directorios importantes:

    - cutext_results: carpeta que incluye los resultados de la ejecución de CUTEXT.
    - cutext_corpus: carpeta que alberga el corpus elaborado al procesar los resultados de CUTEXT.

In [ ]:
# utilizar el path donde ubican los textos (3.3.2) y generar una lista con el contenido de la misma
l_textos = os.listdir(textos)
# path_textos contiene el path absoluto de los textos
path_textos = []
for file in l_textos:
    path_textos.append(os.path.join(textos, file))

    - 4.2 Extracción de terminología mediante CUTEXT: getCutextResults

In [ ]:
# path a la carpeta in de cutext
cutext_in = 'path_to_cutext_in_folder'
# path del output generado por cutext
out = 'path_to_cutext_output_folder'
# El output de cutext se almacenará en cutext_results
# ejecución de la función que integra la funcionalidad de cutext
nimgenetics.getCutextResults(cutext_in=cutext_in, textos_ori=path_textos, old=out, new=cutext_results)

        - 4.2.1 Elaboración de un corpus final a partir de los resultados Cutext: cutextCorpus

In [ ]:
# utilizar el contenido de cutext_results para construir una lista cutext
cutext = os.listdir(cutext_results)
# path absoluto de los resultados de cutext
l_cutext = []
for file in cutext:
    l_cutext.append(os.path.join(cutext_results, file))

# ejecución del programa que genera el corpus de cutext
# El resultado se almacena en cutext_corpus
nimgenetics.cutextCorpus(lista_cutext=l_cutext, path_to_words=cutext_corpus)